In [219]:
import pandas as pd
import numpy as np
from collections import defaultdict
import math

# Добавим новых фичей

In [220]:
# Считываю датасет
dataset = pd.read_excel("../datasets/data_weather.xlsx", index_col=[0])

del dataset["R24(мм)"]

Разобью dep_datetime и arr_datetime на dep_date, dep_time, arr_date, arr_time.

In [221]:
dataset['dep_date'] = [pd.Timestamp(str(item).split()[0]) for item in dataset['dep_datetime']]
dataset['dep_time'] = [pd.to_datetime(str(item).split()[1], format='%H:%M:%S').time() if len(str(item).split()) > 1 else np.nan for item in dataset['dep_datetime']]
dataset['arr_date'] = [pd.Timestamp(str(item).split()[0]) for item in dataset['arr_datetime']]
dataset['arr_time'] = [pd.to_datetime(str(item).split()[1], format='%H:%M:%S').time() if len(str(item).split()) > 1 else np.nan for item in dataset['arr_datetime']]

del dataset['dep_datetime']
del dataset['arr_datetime']

In [222]:
dataset

,center,flight_id,uav_type,duration_min,takeoff_lat,takeoff_lon,landing_lat,landing_lon,wind_dir,wind_speed,Т(С),f(%),R(мм),dep_date,dep_time,arr_date,arr_time
22983,Тюмень,7.771444e+09,BLA,960.0,60.400000,69.816667,60.400000,69.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00
23004,Тюмень,7.771445e+09,BLA,960.0,61.100000,72.766667,61.100000,72.766667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00
23003,Тюмень,7.771445e+09,BLA,960.0,60.950000,73.000000,60.950000,73.000000,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00
23002,Тюмень,7.771445e+09,BLA,960.0,61.033333,73.866667,61.033333,73.866667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00
22999,Тюмень,7.771445e+09,BLA,960.0,60.816667,72.816667,60.816667,72.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127695,Екатеринбург,7.772737e+09,25BLA,NaN,57.233333,54.216667,57.233333,54.216667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-13,13:26:00
127705,Екатеринбург,7.772737e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-10,17:25:00
127800,Екатеринбург,7.772742e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-12,01:05:00
128210,Екатеринбург,7.772765e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-18,17:40:00


In [223]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128844 entries, 22983 to 128280
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   center        128844 non-null  object        
 1   flight_id     120031 non-null  float64       
 2   uav_type      128685 non-null  object        
 3   duration_min  117773 non-null  float64       
 4   takeoff_lat   126403 non-null  float64       
 5   takeoff_lon   126403 non-null  float64       
 6   landing_lat   126398 non-null  float64       
 7   landing_lon   126398 non-null  float64       
 8   wind_dir      112870 non-null  object        
 9   wind_speed    111556 non-null  float64       
 10  Т(С)          112870 non-null  float64       
 11  f(%)          112870 non-null  float64       
 12  R(мм)         45477 non-null   float64       
 13  dep_date      120351 non-null  datetime64[ns]
 14  dep_time      120351 non-null  object        
 15  arr_date      1198

Добавлю обозначение сезонности (season), буду ориантироваться на arr_date.

In [224]:
def get_season(month):
    if month in [12, 1, 2]: return 'winter'
    elif month in [3, 4, 5]: return 'spring'
    elif month in [6, 7, 8]: return 'summer'
    else: return 'autumn'

In [225]:
dataset["season"] = [get_season(item.month) for item in dataset["arr_date"]]

In [226]:
dataset

,center,flight_id,uav_type,duration_min,takeoff_lat,takeoff_lon,landing_lat,landing_lon,wind_dir,wind_speed,Т(С),f(%),R(мм),dep_date,dep_time,arr_date,arr_time,season
22983,Тюмень,7.771444e+09,BLA,960.0,60.400000,69.816667,60.400000,69.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter
23004,Тюмень,7.771445e+09,BLA,960.0,61.100000,72.766667,61.100000,72.766667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter
23003,Тюмень,7.771445e+09,BLA,960.0,60.950000,73.000000,60.950000,73.000000,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter
23002,Тюмень,7.771445e+09,BLA,960.0,61.033333,73.866667,61.033333,73.866667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter
22999,Тюмень,7.771445e+09,BLA,960.0,60.816667,72.816667,60.816667,72.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127695,Екатеринбург,7.772737e+09,25BLA,NaN,57.233333,54.216667,57.233333,54.216667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-13,13:26:00,summer
127705,Екатеринбург,7.772737e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-10,17:25:00,summer
127800,Екатеринбург,7.772742e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-12,01:05:00,summer
128210,Екатеринбург,7.772765e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-18,17:40:00,summer


In [227]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128844 entries, 22983 to 128280
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   center        128844 non-null  object        
 1   flight_id     120031 non-null  float64       
 2   uav_type      128685 non-null  object        
 3   duration_min  117773 non-null  float64       
 4   takeoff_lat   126403 non-null  float64       
 5   takeoff_lon   126403 non-null  float64       
 6   landing_lat   126398 non-null  float64       
 7   landing_lon   126398 non-null  float64       
 8   wind_dir      112870 non-null  object        
 9   wind_speed    111556 non-null  float64       
 10  Т(С)          112870 non-null  float64       
 11  f(%)          112870 non-null  float64       
 12  R(мм)         45477 non-null   float64       
 13  dep_date      120351 non-null  datetime64[ns]
 14  dep_time      120351 non-null  object        
 15  arr_date      1198

Добавлю показатель загруженности области (air_traffic_load)

In [228]:
region_area_by_city = {
    "Тюмень": 1464173,            
    "Москва": 2561.5,             
    "Красноярск": 2_366_797,      
    "Магадан": 462_464,           
    "Якутск": 3_083_523,          
    "Иркутск": 774_846,           
    "Новосибирск": 177_756,       
    "Ростов": 100_967,            
    "Екатеринбург": 194_307,      
    "Калининград": 15_125,        
    "Санкт-Петербург": 1_403,     
    "Самара": 53_565,              
    "Хабаровск": 787_633,          
    "Симферополь": 26_081         
}

In [229]:
traffic_cnt = dataset.groupby('center').count()["season"]
air_traffic_load = defaultdict(int)

for idx in traffic_cnt.index:
    air_traffic_load[idx] = traffic_cnt[idx]/region_area_by_city[idx]

air_traffic_load = pd.DataFrame(list(air_traffic_load.items()), columns=["center", "air_traffic_load"])

dataset = dataset.merge(air_traffic_load, on="center", how="left")

In [230]:
dataset

,center,flight_id,uav_type,duration_min,takeoff_lat,takeoff_lon,landing_lat,landing_lon,wind_dir,wind_speed,Т(С),f(%),R(мм),dep_date,dep_time,arr_date,arr_time,season,air_traffic_load
0,Тюмень,7.771444e+09,BLA,960.0,60.400000,69.816667,60.400000,69.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935
1,Тюмень,7.771445e+09,BLA,960.0,61.100000,72.766667,61.100000,72.766667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935
2,Тюмень,7.771445e+09,BLA,960.0,60.950000,73.000000,60.950000,73.000000,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935
3,Тюмень,7.771445e+09,BLA,960.0,61.033333,73.866667,61.033333,73.866667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935
4,Тюмень,7.771445e+09,BLA,960.0,60.816667,72.816667,60.816667,72.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128839,Екатеринбург,7.772737e+09,25BLA,NaN,57.233333,54.216667,57.233333,54.216667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-13,13:26:00,summer,0.048660
128840,Екатеринбург,7.772737e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-10,17:25:00,summer,0.048660
128841,Екатеринбург,7.772742e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-12,01:05:00,summer,0.048660
128842,Екатеринбург,7.772765e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-18,17:40:00,summer,0.048660


Можно добавить и само количество полетов в регионе (traffic_cnt)

In [231]:
dataset = dataset.merge(traffic_cnt.to_frame(name="traffic_cnt"), left_on="center", right_index=True)

In [232]:
dataset

,center,flight_id,uav_type,duration_min,takeoff_lat,takeoff_lon,landing_lat,landing_lon,wind_dir,wind_speed,Т(С),f(%),R(мм),dep_date,dep_time,arr_date,arr_time,season,air_traffic_load,traffic_cnt
0,Тюмень,7.771444e+09,BLA,960.0,60.400000,69.816667,60.400000,69.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901
1,Тюмень,7.771445e+09,BLA,960.0,61.100000,72.766667,61.100000,72.766667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901
2,Тюмень,7.771445e+09,BLA,960.0,60.950000,73.000000,60.950000,73.000000,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901
3,Тюмень,7.771445e+09,BLA,960.0,61.033333,73.866667,61.033333,73.866667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901
4,Тюмень,7.771445e+09,BLA,960.0,60.816667,72.816667,60.816667,72.816667,СЗ,2.166667,-12.216667,72.5,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128839,Екатеринбург,7.772737e+09,25BLA,NaN,57.233333,54.216667,57.233333,54.216667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-13,13:26:00,summer,0.048660,9455
128840,Екатеринбург,7.772737e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-10,17:25:00,summer,0.048660,9455
128841,Екатеринбург,7.772742e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-12,01:05:00,summer,0.048660,9455
128842,Екатеринбург,7.772765e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2025-07-18,17:40:00,summer,0.048660,9455


In [233]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128844 entries, 0 to 128843
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   center            128844 non-null  object        
 1   flight_id         120031 non-null  float64       
 2   uav_type          128685 non-null  object        
 3   duration_min      117773 non-null  float64       
 4   takeoff_lat       126403 non-null  float64       
 5   takeoff_lon       126403 non-null  float64       
 6   landing_lat       126398 non-null  float64       
 7   landing_lon       126398 non-null  float64       
 8   wind_dir          112870 non-null  object        
 9   wind_speed        111556 non-null  float64       
 10  Т(С)              112870 non-null  float64       
 11  f(%)              112870 non-null  float64       
 12  R(мм)             45477 non-null   float64       
 13  dep_date          120351 non-null  datetime64[ns]
 14  dep_

Рассчитаю пройденный путь (м) (distance) и скорость дрона (speed) (м/с)

In [234]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли в км
    # переводим в радианы
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.asin(math.sqrt(a))

    return R * c

In [235]:
dataset["distance"] = [
    haversine(
        dataset.loc[i, "takeoff_lat"],
        dataset.loc[i, "takeoff_lon"],
        dataset.loc[i, "landing_lat"],
        dataset.loc[i, "landing_lon"]
    ) * 1000 for i in range(dataset.shape[0])
]

dataset["speed"] = dataset["distance"] / dataset["duration_min"]

In [238]:
dataset

,center,flight_id,uav_type,duration_min,takeoff_lat,takeoff_lon,landing_lat,landing_lon,wind_dir,wind_speed,...,R(мм),dep_date,dep_time,arr_date,arr_time,season,air_traffic_load,traffic_cnt,distance,speed
0,Тюмень,7.771444e+09,BLA,960.0,60.400000,69.816667,60.400000,69.816667,СЗ,2.166667,...,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901,0.000000,0.0
1,Тюмень,7.771445e+09,BLA,960.0,61.100000,72.766667,61.100000,72.766667,СЗ,2.166667,...,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901,0.000000,0.0
2,Тюмень,7.771445e+09,BLA,960.0,60.950000,73.000000,60.950000,73.000000,СЗ,2.166667,...,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901,0.000000,0.0
3,Тюмень,7.771445e+09,BLA,960.0,61.033333,73.866667,61.033333,73.866667,СЗ,2.166667,...,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901,0.000000,0.0
4,Тюмень,7.771445e+09,BLA,960.0,60.816667,72.816667,60.816667,72.816667,СЗ,2.166667,...,0.0,2024-01-01,00:00:00,2024-01-01,16:00:00,winter,0.027935,40901,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128839,Екатеринбург,7.772737e+09,25BLA,NaN,57.233333,54.216667,57.233333,54.216667,NaN,NaN,...,NaN,NaT,NaN,2025-07-13,13:26:00,summer,0.048660,9455,0.000000,NaN
128840,Екатеринбург,7.772737e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,...,NaN,NaT,NaN,2025-07-10,17:25:00,summer,0.048660,9455,9266.243887,NaN
128841,Екатеринбург,7.772742e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,...,NaN,NaT,NaN,2025-07-12,01:05:00,summer,0.048660,9455,9266.243887,NaN
128842,Екатеринбург,7.772765e+09,AER,NaN,57.400000,56.866667,57.483333,56.866667,NaN,NaN,...,NaN,NaT,NaN,2025-07-18,17:40:00,summer,0.048660,9455,9266.243887,NaN


In [241]:
dataset.to_excel("../datasets/data_new_features.xlsx")